In [ ]:
import sys
!{sys.executable} -m pip install pandas seaborn matplotlib numpy

Defaulting to user installation because normal site-packages is not writeable
  Using cached pandas-1.3.3-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (11.5 MB)
  Using cached seaborn-0.11.2-py3-none-any.whl (292 kB)
  Using cached matplotlib-3.4.3-cp39-cp39-manylinux1_x86_64.whl (10.3 MB)
     |███████████████████████▊        | 11.7 MB 423 kB/s eta 0:00:10

# Formatter di dati post raccolta

In [15]:
import networkx as nx
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import json
import numpy as np

%run Utilyties.ipynb

data = read_data('../data/extraction/data_complete.json')
data_user = read_data('../data/final/final_user_classification.json')

### Grado di classificazione degli hashtag di un tweet

Ogni **hashtag** ha un *type* che identifica un singolo tweet in una categoria

In [16]:
h_classification = read_data('../data/final/hashtags_classification.json')

### Definizione della classificazione di un tweet

in base al totale degli hashtag e delle classificazioni relative ad ognuni di questi si identifica a che categoria appartiene in tweet

In [17]:
def force_tweet_classification(tweet, types):
    lst = []
    if types == "p":
        lst = ["robertosaviano", "ProfCampagna", "pietroraffa", "LuciaCaprig", "manginobrioches"]
    elif types == "pc":
        lst = ["PBerizzi", "MarcoNoel19"]
    elif types == "cp":
        lst = ["lefrasidiosho", "LeonardoPanetta"]
        
    find = False
    for l in lst:
        if (l == tweet["user"]) or (l in tweet["retweets"]):
            find = True
    return find

In [18]:
def addClassification(tweet):
    if force_tweet_classification(tweet, "p"):
        return -3.0
    elif force_tweet_classification(tweet, "pc"):
        return -1.0
    elif force_tweet_classification(tweet, "cp"):
        return 1.0
    elif force_tweet_classification(tweet, "c"):
        return 3.0
    else:
        hashtags = tweet['hashtags']
        tmp = []
        classification = 0
        h_not_0 = 0
        for h_group in h_classification:
            for h_tweet in hashtags:
                if (h_tweet.lower() == h_group['hashtag']):
                    if h_group['type'] != 0:
                        h_not_0 += 1
                    classification += h_group['type']

        if h_not_0 == 0:
            return 0
        else:
            return classification/h_not_0

In [19]:
def addUserClassification(user):
    for u in data_user:
        if u['user'] == user:
            return u['classification'], u['vip']


### Cercare i valori che ci servono in base ad una chiave chiave 

Questa funzione **\__init__** inizializza un oggetto e una lista vuota.

- L'oggetto identifica un singolo tweet estratto grazie all'API. 
- La lista vuota rappresenterà il nostro output

La funzione **search_key** prende come parametri il self, una chiave e un type:

- la chiave è una stringa che identifica la chiave da cercare in tutto il nostro oggetto(tweet)
- il type invece identifica il tipo di risultato che vogliamo ottenere

La funzione **\__search** genrera il nostro output in base alla chiave e al tipo passati nella search_key 

In [20]:
class GetKeyValue(object):
    def __init__(self, o):
        self.json_object = o

        self.result_list = []

    def search_key(self, key, types):
        self.result_list = []
        self.__search(self.json_object, key, types)
        return list(set(self.result_list))

    def __search(self, json_object, key, types):

        for k in json_object:
            if k == key:
                parent = json_object[k]
                if parent != None:
                    if types == "mentions":
                        for mention in parent:
                            self.result_list.append(mention['username'])
                    if types == "retweeted" or types == "quoted":
                        for retweet in parent:
                            if retweet['type'] == types and "author" in retweet:
                                self.result_list.append(retweet['author']['username'])
                    if types == "hashtags":
                        for hashtag in parent:
                            self.result_list.append(hashtag['tag'])
                    if types == "reply":
                        if parent != {}:
                            self.result_list.append(parent['username'])
                                
            if isinstance(json_object[k], dict):
                self.__search(json_object[k], key, types)
            if isinstance(json_object[k], list):
                for item in json_object[k]:
                    if isinstance(item, dict):
                        self.__search(item, key, types)
        return

### Creazione del file json finale

Qua si va a snellire il file json estratto con l'API andando a considerare solo le informazioni utili al nostro scopo

Es di un tweet finale:

`{
   tweet_id: string;
   user: string;
   date: string:
   text: string;
   hashtags: [string]
   mentions: [string]
   reply_to: string;
   quote_to: string;
   followers: int:
   following: int;
   tweets: int;
   tweet_classification: float(-1,0,1);
   user_classification: float(-1,0,1);
   vip: int(0,1,2);
}`

In [21]:
file = open('../data/final/RAW/data_complete_raw.json', 'w', encoding='utf-8')
for tweet in data:
    gkv = GetKeyValue(o=tweet)
    t = {}
    t['tweet_id'] = tweet['id']
    t['user'] = tweet['author']['username']
    t['date'] = tweet['created_at']
    t['text'] = tweet['text']
    t['hashtags'] = gkv.search_key('hashtags', "hashtags")
    t['mentions'] = gkv.search_key('mentions', "mentions")
    t['retweets'] = gkv.search_key('referenced_tweets' , "retweeted")
    t['reply_to'] = gkv.search_key('in_reply_to_user' ,"reply")
    t['quote_to'] = gkv.search_key('referenced_tweets', 'quoted')
    t['followers'] = tweet['author']['public_metrics']['followers_count']
    t['following'] = tweet['author']['public_metrics']['following_count']
    t['tweets'] = tweet['author']['public_metrics']['tweet_count']
    t['tweet_classification'] = addClassification(t)
    t['user_classification'], t['vip'] = addUserClassification(tweet['author']['username'])

    file.write(json.dumps(t))

### Formatter del file RAW in un json corretto

il file creato precedentemente non è fomattato in modo corretto come json, in questo modo il file viene letto e corretto

In [22]:
import json
from operator import itemgetter


with open('../data/final/RAW/data_complete_raw.json', 'r') as f:
    data = f.read()

file = open('../data/final/data_complete.json', 'a', encoding='utf-8')

data = "[" + data.replace("}{", "},{", data.count("}")-1) + "]"

file.write(data)

21820311

### Ordina per tempo i tweet

In [23]:
data = read_data('../data/final/data_complete.json')
    
data.sort(key= lambda x:x['tweet_id'])

file = open('../data/final/data_complete.json', 'w', encoding='utf-8')

file.write(json.dumps(data))

21859208

# Classificazione degli utenti
### !!! run solo una volta per creare la classificazione !!!
 
Carico il file dei tweets

In [10]:
data = read_data('../data/final/data_complete.json')

Ordino la lista dei tweet in base all'utente

In [11]:
data.sort(key=lambda tweet:tweet['user'])

In [12]:
data

[{'tweet_id': '1408766121277263876',
  'user': '00400708020_',
  'date': '2021-06-26T12:36:31.000Z',
  'text': "@EsercitoCrucian Se si inginocchiano spengo la TV è non seguirò mai più l'Italia.\n\n#azzurriday\n#IoNonMiInginocchio \n#FIGC",
  'hashtags': ['FIGC', 'azzurriday', 'AustriaItalia', 'IoNonMiInginocchio'],
  'mentions': ['EsercitoCrucian'],
  'retweets': [],
  'reply_to': ['EsercitoCrucian'],
  'quote_to': [],
  'followers': 7,
  'following': 18,
  'tweets': 171,
  'tweet_classification': 3.0},
 {'tweet_id': '1406659528553345030',
  'user': '007Vincentxxx',
  'date': '2021-06-20T17:05:40.000Z',
  'text': 'RT @fanpage: #Euro2020 #ItaliaGalles, soltanto cinque calciatori azzurri si sono inginocchiati per il #BlackLivesMatter, ma dietro la dimen…',
  'hashtags': ['ItaliaGalles', 'BlackLivesMatter', 'Euro2020'],
  'mentions': ['fanpage'],
  'retweets': ['fanpage'],
  'reply_to': [],
  'quote_to': [],
  'followers': 1117,
  'following': 1078,
  'tweets': 108059,
  'tweet_classifica

### Definizione della classificazione di un utente in base ai suoi tweets

funzione che classifica gli utenti in base a tutti i tweet raccolti di quell'utente

In [13]:
from statistics import mean

def get_user_classification(class_list):
    return mean(class_list) 

scandisco la lista raggruppando ogni oggetto in base all'utente

- k = utente
- v = tweet dell'utente

**print(k, list(v))**

- k [v1,v2]

**esempio di output**: *utente [{tweet1},{tweet2}]*

In [14]:
from itertools import groupby

final = []

for k,v in groupby(data,key=lambda tweet:tweet['user']):
    obj = {}
    tweets = 0
    classifications = []
    for tweet in list(v):
        classifications.append(tweet['tweet_classification'])
        obj['user'] = k
        obj['classification'] = get_user_classification(classifications)
        obj['vip'] = get_vip(tweet['followers'])
        final.append(obj)
file = open('../data/final/final_user_classification.json', 'w', encoding='utf-8') 
f = list({v['user']:v for v in final}.values())
file.write(json.dumps(f))

978414